In [2]:
import pandas as pd
import os
import re

# =========================
# 설정값
# =========================
BASE_PATH = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종"

# 입력: BM_통합파일_추가.xlsx 또는 BM_통합파일_추가_YYYYMM.xlsx 등
INPUT_PATTERN = re.compile(r"^BM_통합파일(_.*)?\.xlsx$", re.IGNORECASE)

# 출력 파일명 Prefix
OUTPUT_PREFIX = "BM 통합_데이터 변환_BM코드 매핑"

# 중간 파일 저장(디버깅용) 여부
SAVE_INTERMEDIATE = False  # True로 바꾸면 컬럼제거 통합파일도 저장
INTERMEDIATE_NAME = "BM_컬럼제거.xlsx"

# BM코드 매핑
BM_MAP = {
    "Barclays Global Aggregate (USD Hedged)": "ZCXK000002",
    "Bloomberg Barclays Global Aggregate - Corp(USD Hedged)": "ZCXK000002",
    "MSCI ACWI NR USD": "ZCXK000001",
    "MSCI EM NR USD": "ZDHK000002",
    "MSCI USA NR USD": "ZDHK000003",
    "MSCI WI NR USD": "ZDHK000001"
}

# =========================
# 유틸: 최신 입력 파일 선택
# =========================
def pick_latest_file(base_path: str, pattern: re.Pattern) -> str:
    files = [f for f in os.listdir(base_path) if pattern.match(f)]
    if not files:
        raise FileNotFoundError("BM_통합파일*.xlsx 파일을 찾을 수 없습니다.")
    files.sort(key=lambda f: os.path.getmtime(os.path.join(base_path, f)), reverse=True)
    return os.path.join(base_path, files[0])

# =========================
# 유틸: 파일명 suffix(기간) 생성
# - 단일: 2025.12
# - 다중(동일연도): 2025.07~12
# - 다중(연도넘김): 2025.11~2026.02
# =========================
def make_suffix_from_sheets(target_sheets: list[str]) -> str:
    if not target_sheets:
        return "(result)"

    start, end = target_sheets[0], target_sheets[-1]
    sy, sm = start[:4], start[4:]
    ey, em = end[:4], end[4:]

    if sy == ey:
        if sm == em:
            return f"({sy}.{sm})"
        return f"({sy}.{sm}~{em})"
    return f"({sy}.{sm}~{ey}.{em})"

# =========================
# 1) 입력 파일 로드 및 대상 시트 선정
# =========================
input_path = pick_latest_file(BASE_PATH, INPUT_PATTERN)
print(f"📂 입력 파일: {input_path}")

excel = pd.ExcelFile(input_path)
all_sheets = excel.sheet_names

target_sheets = [s for s in all_sheets if len(s) == 6 and s.isdigit()]
target_sheets.sort()
print("처리할 시트들:", target_sheets)

# (선택) 중간 산출물 저장용 writer
intermediate_path = os.path.join(BASE_PATH, INTERMEDIATE_NAME)
intermediate_writer = None
if SAVE_INTERMEDIATE:
    intermediate_writer = pd.ExcelWriter(intermediate_path, engine="openpyxl")

# =========================
# 2) 시트별: 첫 행 제거 + EQUITY_RETURN 컬럼 제거 + 변환(melt)
# =========================
final_parts = []

for sheet in target_sheets:
    df_raw = pd.read_excel(input_path, sheet_name=sheet, header=None)

    if df_raw.empty:
        print(f"⚠️ {sheet}: 데이터 없음. 스킵")
        continue

    # ---- (A) 첫 행 제거
    df1 = df_raw.iloc[1:].reset_index(drop=True)
    if df1.empty:
        print(f"⚠️ {sheet}: 첫 행 제거 후 데이터 없음. 스킵")
        continue

    # ---- (B) (제거 후) 새 첫 행 기준으로 EQUITY_RETURN 포함 컬럼 제거
    equity_cols = [col for col in df1.columns if "EQUITY_RETURN" in str(df1.iloc[0, col])]
    if equity_cols:
        df1 = df1.drop(columns=equity_cols)
        print(f"🧹 {sheet}: EQUITY_RETURN 컬럼 {len(equity_cols)}개 제거")

    # ---- (C) 컬럼명 세팅: 첫 행을 헤더로 사용
    # df1의 0행을 컬럼명으로, 나머지를 데이터로
    df1.columns = df1.iloc[0]
    df = df1.iloc[1:].reset_index(drop=True)

    if df.empty:
        print(f"⚠️ {sheet}: 헤더 설정 후 데이터 없음. 스킵")
        continue

    # ---- (D) A열(첫 번째 컬럼) 값이 없으면 행 삭제
    first_col = df.columns[0]
    before_cnt = len(df)
    df = df.dropna(subset=[first_col])
    # 공백 문자열도 제거(필요 시)
    df = df[df[first_col].astype(str).str.strip() != ""]
    removed = before_cnt - len(df)
    if removed > 0:
        print(f"🧹 {sheet}: A열 기준 {removed}행 삭제")

    # ---- (E) 국가 컬럼 체크
    if "국가" not in df.columns:
        print(f"[⚠️ 경고] 시트 {sheet}에 '국가' 컬럼이 없습니다. 건너뜁니다.")
        continue

    # ---- (F) melt로 변환(루프 제거)
    valid_columns = [c for c in df.columns if pd.notna(c) and c != "국가"]
    if not valid_columns:
        print(f"⚠️ {sheet}: 변환할 비중 컬럼이 없습니다. 스킵")
        continue

    melted = df.melt(
        id_vars=["국가"],
        value_vars=valid_columns,
        var_name="자산구분(ASSET_ID)",
        value_name="비중(WGT)"
    )

    # 수치 변환 및 정리
    melted["비중(WGT)"] = pd.to_numeric(melted["비중(WGT)"], errors="coerce")
    melted = melted.dropna(subset=["비중(WGT)"])
    melted = melted[melted["비중(WGT)"] != 0]

    # 메타 컬럼 추가
    melted = melted.rename(columns={"국가": "데이터ID(DATA_ID)"})
    melted["기준일"] = f"{sheet}01"
    melted["구분"] = "FUND"
    melted["기관코드"] = "1061"
    melted["펀드코드"] = ""
    melted["BM코드"] = ""         # 일단 빈값, 아래에서 매핑으로 채움
    melted["데이터구분"] = "BM"

    melted = melted[[
        "기준일","구분","기관코드","펀드코드","BM코드",
        "자산구분(ASSET_ID)","데이터ID(DATA_ID)","데이터구분","비중(WGT)"
    ]]

    final_parts.append(melted)

    # (선택) 중간 컬럼제거 결과를 시트로 저장
    if SAVE_INTERMEDIATE and intermediate_writer is not None:
        # df1(헤더 포함 형태)이 아닌, df(데이터만) 저장을 원하면 df로 바꾸세요.
        # 여기서는 df1(헤더행 포함 제거 후 상태)을 그대로 저장하지 않고,
        # df_raw->정제된 "df1(헤더행 포함)" 형태를 저장하기보다,
        # 최종 df를 저장하는 것이 보통 더 낫습니다.
        df.to_excel(intermediate_writer, sheet_name=sheet, index=False)

# 중간 writer 닫기
if SAVE_INTERMEDIATE and intermediate_writer is not None:
    intermediate_writer.close()
    print(f"📎 중간 파일 저장 완료: {intermediate_path}")

if not final_parts:
    raise ValueError("변환된 데이터가 없습니다. (국가 컬럼/시트 구조/데이터 확인 필요)")

result_df = pd.concat(final_parts, ignore_index=True)

# =========================
# 3) BM코드 매핑
# =========================
result_df["BM코드"] = result_df["자산구분(ASSET_ID)"].map(BM_MAP)

# 데이터ID(DATA_ID) 없는 행 제거(요청 로직 유지)
result_df = result_df.dropna(subset=["데이터ID(DATA_ID)"])

print(result_df.head(10))

# =========================
# 4) 저장(기간 축약 파일명)
# =========================
suffix = make_suffix_from_sheets(target_sheets)
out_name = f"{OUTPUT_PREFIX}_{suffix}.xlsx"
output_path = os.path.join(BASE_PATH, out_name)

result_df.to_excel(output_path, index=False)
print(f"✅ {out_name} 저장 완료: {output_path}")

# (선택) BM코드 미매핑 항목 확인
unmapped = result_df[result_df["BM코드"].isna()]["자산구분(ASSET_ID)"].dropna().unique()
if len(unmapped) > 0:
    print(f"⚠️ BM코드 미매핑 ASSET_ID {len(unmapped)}개(샘플):", unmapped[:20])


📂 입력 파일: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM_통합파일.xlsx
처리할 시트들: ['202510', '202511', '202512']
🧹 202510: EQUITY_RETURN 컬럼 4개 제거
🧹 202510: A열 기준 1행 삭제
🧹 202511: EQUITY_RETURN 컬럼 4개 제거
🧹 202511: A열 기준 1행 삭제
🧹 202512: EQUITY_RETURN 컬럼 4개 제거
🧹 202512: A열 기준 1행 삭제
        기준일    구분  기관코드 펀드코드        BM코드    자산구분(ASSET_ID)  \
0  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
1  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
2  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
3  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
4  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
5  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
6  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
7  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
8  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   
9  20251001  FUND  1061       ZCXK000001  MSCI ACWI NR USD   

           데이터ID(DATA_ID) 데이터